In [35]:
#3RD ANSWER

# Diffie-Hellman Key Exchange
p = 13
g = 5
a = 7  # Alice's secret number
b = 3  # Bob's secret number

def decryption(cipher, key):
    plain = []
    i = 0
    while i < len(cipher):
        char = cipher[i]
        if char.isupper():
            plain.append(chr(((ord(char) - key - 65) % 26) + 65))
        elif char.islower():
            plain.append(chr(((ord(char) - key - 97) % 26) + 97))
        else:
            plain.append(char)
        i += 1

    return ''.join(plain)

# Compute modulo and exchange
A = (g ** a) % p  # Assume A is sent to Bob
B = (g ** b) % p  # Assume B is sent to Alice
# Compute shared secret Key
shared_key_alice = (B ** a) % p
shared_key_bob = (A ** b) % p

print("Shared secret key is (k) :", shared_key_bob)

# Usage of the alternative decryption function:
encrypted_text = "H98A9W_H6UM8W_6A_9_D6C_5ZCI9C8I_D9FF6IFD"
decrypted_text = decryption(encrypted_text, shared_key_bob)
print("Decrypted Plaintext :", decrypted_text)


Shared secret key is (k) : 5
Decrypted Plaintext : C98V9R_C6PH8R_6V_9_Y6X_5UXD9X8D_Y9AA6DAY


In [47]:
#4th Answer

import requests
import hashlib
import itertools

url = "https://raw.githubusercontent.com/um-dsp/CIS545/main/dictionary.txt"
response = requests.get(url)
dictionary_words = response.text.split("\n")

given_hash= "1140ccd48b9dfa0709afe843add51b7b109b6c6cae918854bcc55fb77ebb04005c0d38cdcc7b0c2882aa4e0123051ad9b59fc1894ba853517529e1984fe51292"

# Defining the Hash functions for (MD5, SHA128, SHA224, SHA256, SHA384, SHA512)
def md5_hash(plain_text):
    return hashlib.md5(plain_text.encode()).hexdigest()


def sha1_hash(plain_text):
    return hashlib.sha1(plain_text.encode()).hexdigest()


def sha224_hash(plain_text):
    return hashlib.sha224(plain_text.encode()).hexdigest()


def sha256_hash(plain_text):
    return hashlib.sha256(plain_text.encode()).hexdigest()


def sha384_hash(plain_text):
    return hashlib.sha384(plain_text.encode()).hexdigest()


def sha512_hash(plain_text):
    return hashlib.sha512(plain_text.encode()).hexdigest()


def password_hash_check(password_hash):
    # Cross-checking the computed Hashes with the list of Password Hashes
    # Returns the Hash if found, else returns None.

    for word in dictionary_words:
        reversed_word = word[::-1]
        password = word + reversed_word
        for h1 in (md5_hash, sha1_hash, sha224_hash, sha384_hash, sha512_hash, sha256_hash):
            for h2 in (md5_hash, sha1_hash, sha224_hash, sha384_hash, sha512_hash, sha256_hash):
                for h3 in (md5_hash, sha1_hash, sha224_hash, sha384_hash, sha512_hash, sha256_hash):
                    # Computing the layered Hash function and appending the ASCII value to the password
                    # Since the given Hashes are of length 64 characters, we need SHA 256
                    h4 = sha512_hash(h3(h2(h1(password))))
                    if h4 == password_hash:
                        return word
    return ""



print(password_hash_check(given_hash))

madam


In [15]:
#7TH ANSWER

import random

#We use the theory as follows where for participant X and Y sending their data to servers A and B - XA= rand(0,Q-1) XB =(X-XA)%Q do the same for YA and YB
#fA=f(YA,XA) , fB =f(YB,XB) which gives f=(fA+fB)%Q

# Function to generate random values between a range
def randomize(min_value, max_value):
    return random.randint(min_value, max_value)

# Function for Alice to compute secret shares
def Alice_secrets(Q, A):
    SA = randomize(0, Q - 1)
    SB = (A - SA) % Q
    return SB, SA

# Function for Bob to compute secret shares
def Bob_secrets(Q, B):
    SA = randomize(0, Q - 1)
    SB = (B - SA) % Q
    return SB, SA

# Function for Charles to compute secret shares
def Charles_secrets(Q, C):
    SA = randomize(0, Q - 1)
    SC = (C - SA) % Q
    return SC, SA

# Alice, Bob, and Charles define their private salaries
A = 50000  # Alice's salary
B = 60000  # Bob's salary
C = 70000  # Charles's salary

# Prime number larger than the field of all the above salaries
Q = 199999

# when Alice, Bob, and Carol compute the value without the others knowing their actual salaries
Alice_values = Alice_secrets(Q, A)
Bob_values = Bob_secrets(Q, B)
Charles_values = Charles_secrets(Q, C)

SA = Alice_values[1] + Bob_values[1] + Charles_values[1]
SB = Alice_values[0] + Bob_values[0] + Charles_values[0]

average_salary = ((SA + SB) % Q) / 3

print("Average Salary computed using additive secret sharing:", average_salary)


Average Salary computed using additive secret sharing: 60000.0


In [30]:
#8TH ANSWER

from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding


class Aggregator:
    def __init__(self):
        self.__private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
        )
        self.public_key = self.__private_key.public_key()

    def decipher(self, ciphertext):
        return self.__private_key.decrypt(
            ciphertext,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

    def custom_verify_sign_signature(self, signature, data, signer_public_key):
        try:
            signer_public_key.verify(
                signature,
                data,
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return True
        except Exception as e:
            print(f"Signature verification failed for aggregator: {e}")
            return False

    def encipher(self, data, recipient_public_key):
        encrypted_data = recipient_public_key.encrypt(
            data,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        print(f"Encrypted data for Aggregator: {encrypted_data.hex()}")
        return encrypted_data


class Participant:
    def __init__(self, name, private_info):
        self.name = name
        self.private_info = private_info
        self.__private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
        )
        self.public_key = self.__private_key.public_key()

    def decipher(self, ciphertext):
        return self.__private_key.decrypt(
            ciphertext,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

    def verify_sign(self, data):
        return self.__private_key.sign(
            data,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )

    def custom_verify_signature(self, signature, data, signer_public_key):
        try:
            signer_public_key.verify(
                signature,
                data,
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return True
        except Exception as e:
            print(f"Signature verification failed for {self.name}: {e}")
            return False

    def encipher(self, data, recipient_public_key):
        encrypted_data = recipient_public_key.encrypt(
            data,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        print(f"Encrypted data for {self.name}: {encrypted_data.hex()}")
        return encrypted_data


# Example usage:
private_info_p1 = "Private dataset for P1"
private_info_p2 = "Private dataset for P2"
private_info_p3 = "Private dataset for P3"

participant1 = Participant("P1", private_info_p1)
participant2 = Participant("P2", private_info_p2)
participant3 = Participant("P3", private_info_p3)
aggregator = Aggregator()
# Simulating federated learning process
# Encrypting private datasets and sending to the aggregator
encrypted_info_p1 = participant1.encipher(participant1.private_info.encode(), aggregator.public_key)
encrypted_info_p2 = participant2.encipher(participant2.private_info.encode(), aggregator.public_key)
encrypted_info_p3 = participant3.encipher(participant3.private_info.encode(), aggregator.public_key)

# Aggregator decrypting and combining the datasets
decrypted_info_p1 = aggregator.decipher(encrypted_info_p1).decode()
decrypted_info_p2 = aggregator.decipher(encrypted_info_p2).decode()
decrypted_info_p3 = aggregator.decipher(encrypted_info_p3).decode()

print("Decrypted Info from P1:", decrypted_info_p1)
print("Decrypted Info from P2:", decrypted_info_p2)
print("Decrypted Info from P3:", decrypted_info_p3)

# Sign and verify data integrity
data_to_sign = b"Combined data for training"
signature_p1 = participant1.verify_sign(data_to_sign)
signature_p2 = participant2.verify_sign(data_to_sign)
signature_p3 = participant3.verify_sign(data_to_sign)

# Aggregator verifies the signatures
verification_result_p1 = aggregator.custom_verify_sign_signature(signature_p1, data_to_sign, participant1.public_key)
verification_result_p2 = aggregator.custom_verify_sign_signature(signature_p2, data_to_sign, participant2.public_key)
verification_result_p3 = aggregator.custom_verify_sign_signature(signature_p3, data_to_sign, participant3.public_key)

print("Signature Result for P1:", verification_result_p1)
print("Signature Result for P2:", verification_result_p2)
print("Signature Result for P3:", verification_result_p3)

Encrypted data for P1: 530df47b278788bad2cc17f51c010a7d2c6edcaf3578f4f8f74d8f8ccd7e4333cf8d135005dc73cf24bb70d7f11370cb0793351a542deb33e9302593d11c48e46df3976f3a527702cc8a987563879dd4c3cad3ac3ed8982cdccfbffd5cdf7355deb4ba09c7cbdaccc776583e2ecebe75a02c065ed81dbde09ca4338e173d56cd95ae0d4dca546c8da87ad4a78b49900b9f77c4ba4660fc899e034a5bd809656426c2a1bb6ad4eb481a27197e74cf7cf63727f5d712bc135051ea292512cb047f4d03e73d756c29276ae7e2feea47c3d7ad3b34bc3188943c33a8274571849aa2d303fccfc303293a616ae65514bbb77ebbef1fa68eeaeddb2f6a7f8cc2d7879c
Encrypted data for P2: 9fd07b1465be0bb179c557ab7e786265bfbd94a927b56c6904705970aa2242cae2848bebb5d779e4ac086d8ad9a92b976263cd864716c70d3296fa4f23cbd95cf8f3bfc622efcc98409e0f2917ebd9d661932cc41aa08a02dfa37468d3fcd690c51f17fd0cdfd9e757f6be4169c542147359fc54ae714803f81757f8f76f0210c26223d6cb4b56c1464f1fcde10fdd672f2099d395605a751222af316c154d880fcf155a290bbcdbd7a01d42ece0e4cfddb5025d025d4dd70371ae35d0a2bfb7c826c4b9ac8b2cf51cedfd50927111fe4b541c61c587e0f33df20a0f9